In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.preprocessing import StandardScaler

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import palettable
import matplotlib.pyplot as plt

import yfinance as yf
# from pandas_datareader import data as pdr

import sys
sys.path.insert(0, '../..')
sys.path.insert(0, '../../utils')

#### Import Custom Modules ####

In [ ]:
prj_path = '/Users/darien/Library/Mobile Documents/com~apple~CloudDocs/Code/QuantTrading/TradingProject/TradingLibADS'
sys.path.append(prj_path)

# from utils import DataUtils
# from utils import TickerEnums as te
# from utils.TickerEnums import *
from utils.data_downloader import DataUtils
import utils.analyser as an
import utils.stock_visualizer as sv

In [ ]:
selected_stock_basket = [
    'GE',
    'HON',
    'DE',
    'UNP',
    'FDX',
    'JPM',
    'BAC',
    'WFC',
    'C',
    'GS',
    'MS',
    'AXP',
    'BLK',
    'AAPL',
    'GOOGL',
    'INTC',
    'CRM'
 ]

data_utils = DataUtils(verbose=False)

start_date = '2021-01-01'
end_date = '2024-03-01'

raw_data = data_utils.get_data(tickers=selected_stock_basket,
                               start_date=start_date,
                               end_date=end_date,
                               price_type='Close')

raw_data
df_all = raw_data.copy()

Pulled ['Close'] for ['GE', 'HON', 'DE', 'UNP', 'FDX', 'JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'AXP', 'BLK', 'AAPL', 
'GOOGL', 'INTC', 'CRM'] from 2021-01-01 to 2024-03-01

# Sorting and Ranking Pairs for Trading Strategy
---
## Enhanced Analysis of Correlation and Cointegration
___
This section extends the analysis of stock pairs by sorting and ranking them based on their correlation and cointegration values. This approach is designed to identify the most promising pairs for a trading strategy.

### Sorting Pairs by Correlation and Cointegration
The `sort_pairs` function is a versatile tool for sorting pairs in a dataframe. It first sets the diagonal values to NaN to exclude self-pairs. Then, it applies a mask to focus on the upper triangle of the matrix, ensuring that each pair is considered only once. The function finally unstacks the dataframe and sorts the pairs based on the specified criterion (correlation or cointegration values).

### Ranking Pairs for Trading Opportunities
After sorting, the pairs are ranked based on their cointegration p-values and correlation coefficients. This ranking is crucial to prioritize pairs that show both strong cointegration (suggesting a long-term relationship) and high correlation (indicating short-term alignment).

#### Creating a Combined Ranking
The next step involves merging the ranked cointegration and correlation dataframes. This merged dataframe includes ranks for both cointegration p-values and correlation coefficients. A combined rank is calculated by summing these individual ranks, and an average rank is derived to balance between cointegration and correlation.

---

#### Function: `sort_pairs`

The `sort_pairs` function efficiently sorts pairs in a dataframe based on the given criterion. It handles both correlation and cointegration matrices, making it an essential tool in this analysis. The function is then applied to both the correlation and cointegration dataframes to obtain sorted lists of pairs.

The following code snippet showcases the use of this function and the subsequent ranking process, culminating in a sorted and ranked dataframe that identifies the most promising stock pairs for pairs trading strategies. The final output includes the top 20 pairs, offering a focused list for potential trading opportunities.



In [ ]:
dfs = []
tkr_set = [SP500Stocks]
for ticker_set in tkr_set:
    print(ticker_set)
    data = get_data(ticker_set._member_names_)
    dfs.append(data)
    print(data.head(2))
    print()

df_all = pd.concat(dfs, axis=1).drop_duplicates()
df_all

In [ ]:
def create_dataset(data, look_back, target_col="spread_close", split_sizes=[0.7, 0.2, 0.1], drop_cols: list = []):
    data = data.copy().dropna()
    y = data[target_col]
    y_shifted = y.shift(-look_back)
    data["target"] = y_shifted
    data = data.dropna().drop(drop_cols, axis=1)
    train_size = int(len(data) * split_sizes[0])
    val_size = int(len(data) * split_sizes[1])
    test_size = len(data) - train_size - val_size

    print(
        "---Dataset: Size, Date Cutoff--- \n Train: {}, {} \n Val:   {}, {} \n Test:  {},  {}".format(train_size, data.index[train_size].date(),val_size, data.index[-val_size].date(),test_size, data.index[-test_size].date(),))

    train_data = data.iloc[:train_size]
    val_data = data.iloc[train_size : train_size + val_size]
    test_data = data.iloc[-test_size:]
    return train_data, val_data, test_data


train_data, val_data, test_data = create_dataset(
    pair_data, look_back=5, target_col="spread_close", split_sizes=[0.7, 0.2, 0.1]
)

train_data.head(2)

---Dataset: Size, Date Cutoff--- 
 Train: 546, 2023-04-18 
 Val:   156, 2023-08-09 
 Test:  78,  2023-11-29


,S2_open,S1_open,S2_high,S1_high,S2_low,S1_low,S2_close,S1_close,S2_adj close,S1_adj close,...,S2_ema,S1_macd,S2_macd,S1_dlr,S2_dlr,spread_close,spread_open,spread_high,spread_low,target
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-16,310.000000,204.860001,312.920013,205.570007,309.220001,201.880005,311.959991,203.440002,287.484192,190.686661,...,296.907798,-1.174375,7.167962,-0.063883,1.824462,17.000179,13.039232,14.908616,17.689315,21.930414
2021-02-17,310.029999,201.360001,314.980011,202.190002,308.760010,199.860001,312.739990,201.899994,288.203064,189.243195,...,299.018757,-1.132337,7.795040,-0.759864,0.249720,20.012977,18.142757,21.868550,20.146369,24.675759


In [ ]:
df = raw_data.copy()
# rename index to index
df.index.name = 'index'
DATA = df.copy()
df.corr()


Ticker,AAPL,AXP,BAC,BLK,C,CRM,DE,FDX,GE,GOOGL,GS,HON,INTC,JPM,MS,UNP,WFC
Ticker,,,,,,,,,,,,,,,,,
AAPL,1.000000,0.531576,-0.299144,-0.076802,-0.503730,0.256794,0.337718,0.033327,0.641856,0.575225,0.082628,-0.365208,-0.315341,0.215919,0.107519,0.317492,0.305517
AXP,0.531576,1.000000,0.331082,0.384742,0.095503,0.441622,0.377289,0.105726,0.465789,0.609868,0.576200,0.109514,0.040215,0.529539,0.573314,0.601928,0.778306
BAC,-0.299144,0.331082,1.000000,0.749378,0.782305,0.400026,-0.137043,0.210853,-0.345983,0.363355,0.634208,0.555558,0.611483,0.417289,0.669486,0.587497,0.689683
BLK,-0.076802,0.384742,0.749378,1.000000,0.782594,0.750869,-0.098311,0.559256,0.135301,0.551805,0.834400,0.801651,0.653302,0.771860,0.693887,0.473985,0.535761
C,-0.503730,0.095503,0.782305,0.782594,1.000000,0.561913,-0.364664,0.575900,-0.163256,0.231835,0.517933,0.751753,0.858337,0.541321,0.425740,0.306925,0.291127
CRM,0.256794,0.441622,0.400026,0.750869,0.561913,1.000000,-0.273827,0.652181,0.520781,0.762885,0.571426,0.455298,0.645531,0.839030,0.360083,0.433502,0.341636
DE,0.337718,0.377289,-0.137043,-0.098311,-0.364664,-0.273827,1.000000,-0.231359,0.312484,-0.043045,0.201524,0.045894,-0.440527,0.092443,0.263287,0.066634,0.284361
FDX,0.033327,0.105726,0.210853,0.559256,0.575900,0.652181,-0.231359,1.000000,0.410463,0.465113,0.251538,0.483493,0.695779,0.621037,0.088787,0.252389,0.037039
GE,0.641856,0.465789,-0.345983,0.135301,-0.163256,0.520781,0.312484,0.410463,1.000000,0.461406,0.228088,0.028296,-0.011009,0.651532,0.000867,0.104807,0.103685


In [ ]:
def find_cointegrated_pairs(data):
    """
    This function performs a cointegration test on each pair of stocks in the given dataset.
    It returns the score matrix, p-value matrix, and the pairs of stocks that have a p-value less than 0.05.
    """
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i + 1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs


def calculate_cointegration(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to calculate cointegration of each pair in a dataframe.
    """
    # Initialize an empty DataFrame to store the p-values
    cointegration_df = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    # Calculate the cointegration for each pair
    for i in df.columns:
        for j in df.columns:
            if i != j:
                _, pvalue, _ = coint(df[i], df[j])
                cointegration_df.loc[i, j] = pvalue

    return cointegration_df


# Copy the dataframe and drop NaN values
dff = DATA.copy().dropna()
# dff = df_all.copy().dropna()

# Find cointegrated pairs
scores, pvalues, pairs = find_cointegrated_pairs(dff)
print(pairs)

# Calculate correlation matrix
corr_df = dff.corr()

# Create a dataframe for p-values
pvals_df = pd.DataFrame(pvalues, columns=dff.columns, index=dff.columns)

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(pvals_df, dtype=bool))

# Apply the mask to the dataframe and filter p-values <= 0.2
aggregate_df_masked = pvals_df.where(mask).where(pvals_df <= 0.2)

# Display the dataframe with style
display(
    aggregate_df_masked.style.background_gradient(cmap="RdYlGn", axis=None).set_caption(
        "Cointegration P-Value Matrix"
    )
)

# Set the diagonal to NaN
np.fill_diagonal(corr_df.values, np.nan)

# Apply the mask to the dataframe and filter correlation >= 0.75 or <= -0.75
aggregate_df_masked = corr_df.where(mask).where((corr_df >= 0.75) | (corr_df <= -0.75))

# Display the dataframe with style
display(
    aggregate_df_masked.style.background_gradient(
        cmap="RdYlGn_r", axis=None
    ).set_caption("Correlation Matrix")
)

[('BLK', 'GS'), ('CRM', 'JPM'), ('DE', 'FDX'), ('DE', 'GE'), ('DE', 'GOOGL'), ('DE', 'GS'), ('DE', 'HON'), ('DE', 'INTC'), ('DE', 'JPM'), ('DE', 'MS'), ('DE', 'UNP'), ('GS', 'HON'), ('UNP', 'WFC')]


Ticker,AAPL,AXP,BAC,BLK,C,CRM,DE,FDX,GE,GOOGL,GS,HON,INTC,JPM,MS,UNP,WFC
Ticker,,,,,,,,,,,,,,,,,
AAPL,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
AXP,nan,nan,nan,nan,nan,0.177540,nan,nan,0.185932,nan,nan,nan,nan,nan,nan,0.089600,nan
BAC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
BLK,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.016037,nan,nan,nan,nan,nan,nan
C,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.159814,nan,nan,nan,nan
CRM,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.026538,nan,nan,nan
DE,nan,nan,nan,nan,nan,nan,nan,0.008654,0.022211,0.011775,0.040795,0.015755,0.003088,0.020100,0.044919,0.018292,0.060699
FDX,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Ticker,AAPL,AXP,BAC,BLK,C,CRM,DE,FDX,GE,GOOGL,GS,HON,INTC,JPM,MS,UNP,WFC
Ticker,,,,,,,,,,,,,,,,,
AAPL,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
AXP,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.778306
BAC,nan,nan,nan,nan,0.782305,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
BLK,nan,nan,nan,nan,0.782594,0.750869,nan,nan,nan,nan,0.834400,0.801651,nan,0.771860,nan,nan,nan
C,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.751753,0.858337,nan,nan,nan,nan
CRM,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.762885,nan,nan,nan,0.839030,nan,nan,nan
DE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
FDX,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
sorted_correlation_pairs
# rename both index levels to ticker_0 and ticker_1
sorted_correlation_pairs.index.names = ['ticker_0', 'ticker_1']
sorted_correlation_pairs

ticker_0  ticker_1
INTC      C           0.858337
JPM       CRM         0.839030
MS        GS          0.835230
GS        BLK         0.834400
HON       BLK         0.801651
                        ...   
MS        UNP              NaN
          WFC              NaN
UNP       UNP              NaN
          WFC              NaN
WFC       WFC              NaN
Length: 289, dtype: float64

In [ ]:
def sort_pairs(dataframe, ascending=True):
    """
    Function to sort pairs in a dataframe.
    """
    # Set the diagonal to NaN
    np.fill_diagonal(dataframe.values, np.nan)

    # Create a mask for the upper triangle
    upper_triangle_mask = np.triu(np.ones_like(dataframe, dtype=bool))

    # Unstack and sort the pairs
    unstacked_pairs = dataframe.where(upper_triangle_mask).unstack()
    sorted_pairs = unstacked_pairs.sort_values(ascending=ascending)

    return sorted_pairs


# Use the function
sorted_correlation_pairs = sort_pairs(corr_df, ascending=False)
sorted_correlation_pairs.index.names = ['level_0', 'level_1']
sorted_pvalue_pairs = sort_pairs(pvals_df, ascending=True)
sorted_pvalue_pairs.index.names = ['level_0', 'level_1']


# Create a rank column in sorted_pvalue_pairs
ranked_pvalue_pairs = sorted_pvalue_pairs.reset_index(name="cointeg_pval")
ranked_pvalue_pairs["rank"] = ranked_pvalue_pairs.index

ranked_correlation_pairs = sorted_correlation_pairs.reset_index(name="corr")
ranked_correlation_pairs["rank"] = ranked_correlation_pairs.index

ranked_correlation_cointegration_df = ranked_pvalue_pairs.merge(
    ranked_correlation_pairs, on=["level_0", "level_1"]
).rename(columns={"rank_x": "rank_cointeg_pval", "rank_y": "rank_corr"})
ranked_correlation_cointegration_df["rank_sum"] = (
    ranked_correlation_cointegration_df["rank_cointeg_pval"]
    + ranked_correlation_cointegration_df["rank_corr"]
)
ranked_correlation_cointegration_df["rank_avg"] = (
    ranked_correlation_cointegration_df["rank_sum"] / 2
)  # average the ranks
ranked_correlation_cointegration_df = (
    ranked_correlation_cointegration_df.sort_values(by="rank_sum", ascending=True)
    .sort_index(axis=1)
    .dropna()
    .rename({"level_0": "ticker_0", "level_1": "ticker_1"}, axis=1)
)
# save_path = "/Users/darien/Library/Mobile Documents/com~apple~CloudDocs/Code/TradingStrategy/data/ranked_tickers_corr_cointegration_ALL.csv"

formatted_ranked_correlation_cointegration_df = (
    ranked_correlation_cointegration_df.reset_index()
    .rename_axis(index="agg_rank")[
        [
            "ticker_0",
            "ticker_1",
            "corr",
            "cointeg_pval",
            "rank_corr",
            "rank_cointeg_pval",
            "rank_sum",
            "rank_avg",
        ]
    ]
    .round(4)
)
# formatted_ranked_correlation_cointegration_df.to_csv(save_path, index=False)
formatted_ranked_correlation_cointegration_df

,ticker_0,ticker_1,corr,cointeg_pval,rank_corr,rank_cointeg_pval,rank_sum,rank_avg
agg_rank,,,,,,,,
0,GS,BLK,0.8344,0.0160,3,4,7,3.5
1,JPM,CRM,0.8390,0.0265,1,9,10,5.0
2,INTC,C,0.8583,0.1598,0,19,19,9.5
3,WFC,UNP,0.7002,0.0293,16,10,26,13.0
4,HON,GS,0.6692,0.0188,21,6,27,13.5
...,...,...,...,...,...,...,...,...
131,DE,CRM,-0.2738,0.7958,128,114,242,121.0
132,WFC,GE,0.1037,1.0000,109,134,243,121.5
133,MS,GE,0.0009,1.0000,120,132,252,126.0


### Select Top Pairs

In [ ]:
top_pairs_df = formatted_ranked_correlation_cointegration_df.iloc[:30, :4]
display(top_pairs_df)

top_pairs_tup = [tuple(x) for x in top_pairs_df.iloc[:, :2].values]
# [print(pair) for pair in top_pairs_tup]

top_pairs_list = [i for x in top_pairs_df.iloc[:, :2].values for i in x]
# get rid of duplicates, keep order
top_pairs_list = list(dict.fromkeys(top_pairs_list))
# print(top_pairs_list)

# Plot the top 20 pairs
# cf.plot_tickers_plotly(
#     df[top_pairs_list],
#     convert_to_stationary=True,
#     scale=True,
#     moving_average=True,
#     window_size=40,
#     title="Top 8 REIT Pairs",
#     yaxis_title="Scaled Price",
#     legend_title="Ticker",
#     template='ggplot2'
# )
top_pairs_df.to_csv("../data/ranked_pairs_snp.csv")

,ticker_0,ticker_1,corr,cointeg_pval
agg_rank,,,,
0,GS,BLK,0.8344,0.0160
1,JPM,CRM,0.8390,0.0265
2,INTC,C,0.8583,0.1598
3,WFC,UNP,0.7002,0.0293
4,HON,GS,0.6692,0.0188
5,HON,BLK,0.8017,0.2085
6,JPM,GS,0.7291,0.1579
7,WFC,AXP,0.7783,0.2241
8,WFC,GS,0.7102,0.1923


# Selecting the REITs
---
## Correlation and Cointegration Analysis
___
This section of the code is dedicated to identifying pairs of stocks that are cointegrated. Cointegration is a statistical property of two or more time-series variables. In the context of pairs trading, if two stocks are cointegrated, they move in tandem. Any deviation from this movement can be leveraged to generate a trading signal.

### Definition of Cointegration

Cointegration can be formally defined as follows: Given two non-stationary time series, $X_t$ and $Y_t$, they are said to be cointegrated if there exists a linear combination of them that is stationary. In mathematical terms, if there exists a coefficient $\beta$ such that the time series $Z_t = X_t - \beta Y_t$ is stationary, then $X_t$ and $Y_t$ are cointegrated. The coefficient $\beta$ is often referred to as the cointegration vector.

### Importance of Cointegration in Pairs Trading

The concept of cointegration is crucial in pairs trading as it helps to identify pairs of stocks that maintain a long-term equilibrium relationship, despite their individual prices being non-stationary. This implies that when the spread between the pair (i.e., $Z_t$) deviates from its mean (i.e., the pair is out of equilibrium), there is a high probability that it will revert back to the mean. This reversion presents a potential trading opportunity.

--- 

#### Function: `find_cointegrated_pairs`

The function `find_cointegrated_pairs` conducts a cointegration test on each pair of stocks in the provided dataset. It returns a score matrix, a p-value matrix, and a list of pairs of stocks that have a p-value less than 0.05, indicating significant cointegration.

After defining the function, the code creates a copy of the dataframe and eliminates any rows with NaN values. It then invokes the `find_cointegrated_pairs` function and prints the pairs of stocks that are significantly cointegrated and correlated.

Subsequently, the code computes a correlation matrix for the dataframe and creates a new dataframe for the p-values.

In [ ]:
def find_cointegrated_pairs(data):
    """
    This function performs a cointegration test on each pair of stocks in the given dataset.
    It returns the score matrix, p-value matrix, and the pairs of stocks that have a p-value less than 0.05.
    """
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i + 1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs


def calculate_cointegration(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to calculate cointegration of each pair in a dataframe.
    """
    # Initialize an empty DataFrame to store the p-values
    cointegration_df = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    # Calculate the cointegration for each pair
    for i in df.columns:
        for j in df.columns:
            if i != j:
                _, pvalue, _ = coint(df[i], df[j])
                cointegration_df.loc[i, j] = pvalue

    return cointegration_df


# Copy the dataframe and drop NaN values
dff = DATA.copy().dropna()
# dff = df_all.copy().dropna()

# Find cointegrated pairs
scores, pvalues, pairs = find_cointegrated_pairs(dff)
print(pairs)

# Calculate correlation matrix
corr_df = dff.corr()

# Create a dataframe for p-values
pvals_df = pd.DataFrame(pvalues, columns=dff.columns, index=dff.columns)

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(pvals_df, dtype=bool))

# Apply the mask to the dataframe and filter p-values <= 0.2
aggregate_df_masked = pvals_df.where(mask).where(pvals_df <= 0.2)

# Display the dataframe with style
display(
    aggregate_df_masked.style.background_gradient(cmap="RdYlGn", axis=None).set_caption(
        "Cointegration P-Value Matrix"
    )
)

# Set the diagonal to NaN
np.fill_diagonal(corr_df.values, np.nan)

# Apply the mask to the dataframe and filter correlation >= 0.75 or <= -0.75
aggregate_df_masked = corr_df.where(mask).where((corr_df >= 0.75) | (corr_df <= -0.75))

# Display the dataframe with style
display(
    aggregate_df_masked.style.background_gradient(
        cmap="RdYlGn_r", axis=None
    ).set_caption("Correlation Matrix")
)

[('BLK', 'GS'), ('CRM', 'JPM'), ('DE', 'FDX'), ('DE', 'GE'), ('DE', 'GOOGL'), ('DE', 'GS'), ('DE', 'HON'), ('DE', 'INTC'), ('DE', 'JPM'), ('DE', 'MS'), ('DE', 'UNP'), ('GS', 'HON'), ('UNP', 'WFC')]


Ticker,AAPL,AXP,BAC,BLK,C,CRM,DE,FDX,GE,GOOGL,GS,HON,INTC,JPM,MS,UNP,WFC
Ticker,,,,,,,,,,,,,,,,,
AAPL,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
AXP,nan,nan,nan,nan,nan,0.177540,nan,nan,0.185932,nan,nan,nan,nan,nan,nan,0.089600,nan
BAC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
BLK,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.016037,nan,nan,nan,nan,nan,nan
C,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.159814,nan,nan,nan,nan
CRM,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.026538,nan,nan,nan
DE,nan,nan,nan,nan,nan,nan,nan,0.008654,0.022211,0.011775,0.040795,0.015755,0.003088,0.020100,0.044919,0.018292,0.060699
FDX,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


Ticker,AAPL,AXP,BAC,BLK,C,CRM,DE,FDX,GE,GOOGL,GS,HON,INTC,JPM,MS,UNP,WFC
Ticker,,,,,,,,,,,,,,,,,
AAPL,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
AXP,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.778306
BAC,nan,nan,nan,nan,0.782305,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
BLK,nan,nan,nan,nan,0.782594,0.750869,nan,nan,nan,nan,0.834400,0.801651,nan,0.771860,nan,nan,nan
C,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.751753,0.858337,nan,nan,nan,nan
CRM,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.762885,nan,nan,nan,0.839030,nan,nan,nan
DE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
FDX,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
sorted_correlation_pairs
# rename both index levels to ticker_0 and ticker_1
sorted_correlation_pairs.index.names = ['ticker_0', 'ticker_1']
sorted_correlation_pairs

ticker_0  ticker_1
INTC      C           0.858337
JPM       CRM         0.839030
MS        GS          0.835230
GS        BLK         0.834400
HON       BLK         0.801651
                        ...   
MS        UNP              NaN
          WFC              NaN
UNP       UNP              NaN
          WFC              NaN
WFC       WFC              NaN
Length: 289, dtype: float64

# Sorting and Ranking Pairs for Trading Strategy
---
## Enhanced Analysis of Correlation and Cointegration
___
This section extends the analysis of stock pairs by sorting and ranking them based on their correlation and cointegration values. This approach is designed to identify the most promising pairs for a trading strategy.

### Sorting Pairs by Correlation and Cointegration
The `sort_pairs` function is a versatile tool for sorting pairs in a dataframe. It first sets the diagonal values to NaN to exclude self-pairs. Then, it applies a mask to focus on the upper triangle of the matrix, ensuring that each pair is considered only once. The function finally unstacks the dataframe and sorts the pairs based on the specified criterion (correlation or cointegration values).

### Ranking Pairs for Trading Opportunities
After sorting, the pairs are ranked based on their cointegration p-values and correlation coefficients. This ranking is crucial to prioritize pairs that show both strong cointegration (suggesting a long-term relationship) and high correlation (indicating short-term alignment).

#### Creating a Combined Ranking
The next step involves merging the ranked cointegration and correlation dataframes. This merged dataframe includes ranks for both cointegration p-values and correlation coefficients. A combined rank is calculated by summing these individual ranks, and an average rank is derived to balance between cointegration and correlation.

---

#### Function: `sort_pairs`

The `sort_pairs` function efficiently sorts pairs in a dataframe based on the given criterion. It handles both correlation and cointegration matrices, making it an essential tool in this analysis. The function is then applied to both the correlation and cointegration dataframes to obtain sorted lists of pairs.

The following code snippet showcases the use of this function and the subsequent ranking process, culminating in a sorted and ranked dataframe that identifies the most promising stock pairs for pairs trading strategies. The final output includes the top 20 pairs, offering a focused list for potential trading opportunities.



In [ ]:
def sort_pairs(dataframe, ascending=True):
    """
    Function to sort pairs in a dataframe.
    """
    # Set the diagonal to NaN
    np.fill_diagonal(dataframe.values, np.nan)

    # Create a mask for the upper triangle
    upper_triangle_mask = np.triu(np.ones_like(dataframe, dtype=bool))

    # Unstack and sort the pairs
    unstacked_pairs = dataframe.where(upper_triangle_mask).unstack()
    sorted_pairs = unstacked_pairs.sort_values(ascending=ascending)

    return sorted_pairs


# Use the function
sorted_correlation_pairs = sort_pairs(corr_df, ascending=False)
sorted_correlation_pairs.index.names = ['level_0', 'level_1']
sorted_pvalue_pairs = sort_pairs(pvals_df, ascending=True)
sorted_pvalue_pairs.index.names = ['level_0', 'level_1']


# Create a rank column in sorted_pvalue_pairs
ranked_pvalue_pairs = sorted_pvalue_pairs.reset_index(name="cointeg_pval")
ranked_pvalue_pairs["rank"] = ranked_pvalue_pairs.index

ranked_correlation_pairs = sorted_correlation_pairs.reset_index(name="corr")
ranked_correlation_pairs["rank"] = ranked_correlation_pairs.index

ranked_correlation_cointegration_df = ranked_pvalue_pairs.merge(
    ranked_correlation_pairs, on=["level_0", "level_1"]
).rename(columns={"rank_x": "rank_cointeg_pval", "rank_y": "rank_corr"})
ranked_correlation_cointegration_df["rank_sum"] = (
    ranked_correlation_cointegration_df["rank_cointeg_pval"]
    + ranked_correlation_cointegration_df["rank_corr"]
)
ranked_correlation_cointegration_df["rank_avg"] = (
    ranked_correlation_cointegration_df["rank_sum"] / 2
)  # average the ranks
ranked_correlation_cointegration_df = (
    ranked_correlation_cointegration_df.sort_values(by="rank_sum", ascending=True)
    .sort_index(axis=1)
    .dropna()
    .rename({"level_0": "ticker_0", "level_1": "ticker_1"}, axis=1)
)
# save_path = "/Users/darien/Library/Mobile Documents/com~apple~CloudDocs/Code/TradingStrategy/data/ranked_tickers_corr_cointegration_ALL.csv"

formatted_ranked_correlation_cointegration_df = (
    ranked_correlation_cointegration_df.reset_index()
    .rename_axis(index="agg_rank")[
        [
            "ticker_0",
            "ticker_1",
            "corr",
            "cointeg_pval",
            "rank_corr",
            "rank_cointeg_pval",
            "rank_sum",
            "rank_avg",
        ]
    ]
    .round(4)
)
# formatted_ranked_correlation_cointegration_df.to_csv(save_path, index=False)
formatted_ranked_correlation_cointegration_df

,ticker_0,ticker_1,corr,cointeg_pval,rank_corr,rank_cointeg_pval,rank_sum,rank_avg
agg_rank,,,,,,,,
0,GS,BLK,0.8344,0.0160,3,4,7,3.5
1,JPM,CRM,0.8390,0.0265,1,9,10,5.0
2,INTC,C,0.8583,0.1598,0,19,19,9.5
3,WFC,UNP,0.7002,0.0293,16,10,26,13.0
4,HON,GS,0.6692,0.0188,21,6,27,13.5
...,...,...,...,...,...,...,...,...
131,DE,CRM,-0.2738,0.7958,128,114,242,121.0
132,WFC,GE,0.1037,1.0000,109,134,243,121.5
133,MS,GE,0.0009,1.0000,120,132,252,126.0


### Select Top Pairs

In [ ]:
top_pairs_df = formatted_ranked_correlation_cointegration_df.iloc[:30, :4]
display(top_pairs_df)

top_pairs_tup = [tuple(x) for x in top_pairs_df.iloc[:, :2].values]
# [print(pair) for pair in top_pairs_tup]

top_pairs_list = [i for x in top_pairs_df.iloc[:, :2].values for i in x]
# get rid of duplicates, keep order
top_pairs_list = list(dict.fromkeys(top_pairs_list))
# print(top_pairs_list)

# Plot the top 20 pairs
# cf.plot_tickers_plotly(
#     df[top_pairs_list],
#     convert_to_stationary=True,
#     scale=True,
#     moving_average=True,
#     window_size=40,
#     title="Top 8 REIT Pairs",
#     yaxis_title="Scaled Price",
#     legend_title="Ticker",
#     template='ggplot2'
# )
top_pairs_df.to_csv("../data/ranked_pairs_snp.csv")

,ticker_0,ticker_1,corr,cointeg_pval
agg_rank,,,,
0,GS,BLK,0.8344,0.0160
1,JPM,CRM,0.8390,0.0265
2,INTC,C,0.8583,0.1598
3,WFC,UNP,0.7002,0.0293
4,HON,GS,0.6692,0.0188
5,HON,BLK,0.8017,0.2085
6,JPM,GS,0.7291,0.1579
7,WFC,AXP,0.7783,0.2241
8,WFC,GS,0.7102,0.1923
